In [1]:
!pip install -q -U openai-whisper pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 133.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.


In [3]:
import pandas as pd
import whisper
from pathlib import Path
from tqdm import tqdm
import sys
import os

def process_audio_files(base_dir, model):
    """지정된 기본 디렉토리에서 오디오 파일을 처리하고 텍스트와 레이블을 반환합니다."""

    # pathlib.Path를 사용하여 경로를 객체로 처리합니다.
    base_path = Path(base_dir)
    # 처리할 폴더와 레이블을 매핑합니다.
    dir_label_map = {
        "Phishing": "Phishing",
        "NonPhishing": "NonPhishing"
    }

    all_files_to_process = []

    # 처리해야 할 전체 파일 목록을 먼저 수집합니다.
    # 이렇게 하면 tqdm이 정확한 전체 진행률을 표시할 수 있습니다.
    for dir_name, label in dir_label_map.items():
        folder_path = base_path / dir_name

        if not folder_path.is_dir():
            print(f"경고: '{folder_path}' 폴더를 찾을 수 없습니다. 건너뜁니다.", file=sys.stderr)
            continue

        # .mp3 파일을 재귀적으로 찾지 않고 해당 폴더 내에서만 찾습니다.
        # 만약 하위 폴더까지 모두 검색하려면 .glob("*.mp3") 대신 .rglob("*.mp3")를 사용하세요.
        files = list(folder_path.glob("*.mp3"))
        for file_path in files:
            all_files_to_process.append((file_path, label))

    if not all_files_to_process:
        print("오류: 처리할 .mp3 파일을 찾을 수 없습니다.", file=sys.stderr)
        return []

    results = []

    # tqdm을 적용하여 파일 목록을 순회합니다.
    print(f"총 {len(all_files_to_process)}개의 .mp3 파일 처리를 시작합니다...")
    for file_path, label in tqdm(all_files_to_process, desc="STT 진행률"):
        try:
            # Whisper 모델로 STT 수행
            # file_path를 str으로 변환하여 transcribe 함수에 전달합니다.
            result = model.transcribe(str(file_path))
            text = result["text"]

            results.append({
                "text": text,
                "label": label
            })
        except Exception as e:
            print(f"\n오류: {file_path} 파일 처리 중 문제 발생: {e}", file=sys.stderr)

    return results

def main():
    # --- 설정 ---

    # 1. 오디오 파일이 있는 기본 폴더 경로
    # 예: "C:/my_project/audio_data"
    # 이 폴더 안에 "Phishing", "Non-Phishing" 폴더가 있어야 합니다.
    AUDIO_BASE_DIRECTORY = "/content/drive/MyDrive/Github/2025_Voicephishing/dataset"

    # 2. 출력될 CSV 파일 이름
    OUTPUT_CSV_FILE = "stt_results.csv"

    # 3. Whisper 모델 선택 (예: "tiny", "base", "small", "medium", "large")
    # "base"는 빠르고 적당한 성능을 보입니다.
    MODEL_SIZE = "large"

    # --- 실행 ---

    if not Path(AUDIO_BASE_DIRECTORY).is_dir():
        print(f"오류: '{AUDIO_BASE_DIRECTORY}'는 유효한 디렉토리가 아닙니다. 스크립트를 종료합니다.")
        return

    print(f"Whisper '{MODEL_SIZE}' 모델을 로드합니다. (최초 실행 시 시간이 걸릴 수 있습니다...)")
    try:
        model = whisper.load_model(MODEL_SIZE)
    except Exception as e:
        print(f"Whisper 모델 로드 중 오류 발생: {e}", file=sys.stderr)
        print("ffmpeg가 설치되어 있는지 확인하세요. (https://ffmpeg.org/download.html)", file=sys.stderr)
        return

    # STT 처리 실행
    data = process_audio_files(AUDIO_BASE_DIRECTORY, model)

    if data:
        # 결과를 DataFrame으로 변환
        df = pd.DataFrame(data)

        # CSV 파일로 저장 (encoding='utf-8-sig'는 Excel에서 한글이 깨지지 않게 합니다)
        try:
            df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
            output_full_path = Path(os.getcwd()) / OUTPUT_CSV_FILE
            print(f"\n✅ 완료! 총 {len(df)}개의 데이터가 '{output_full_path}' 파일로 저장되었습니다.")
        except Exception as e:
            print(f"\n오류: CSV 파일 저장 중 문제 발생: {e}", file=sys.stderr)
    else:
        print("\n처리된 데이터가 없어 CSV 파일을 생성하지 않았습니다.")

if __name__ == "__main__":
    main()

Whisper 'large' 모델을 로드합니다. (최초 실행 시 시간이 걸릴 수 있습니다...)
총 80개의 .mp3 파일 처리를 시작합니다...


STT 진행률: 100%|██████████| 80/80 [02:07<00:00,  1.59s/it]


✅ 완료! 총 80개의 데이터가 '/content/stt_results.csv' 파일로 저장되었습니다.


In [9]:
df = pd.read_csv('stt_results.csv')
df['text_length'] = df.text.apply(lambda x: len(x))

In [11]:
df.label.value_counts()

,count
label,
Phishing,40
NonPhishing,40
